# The Truth about Objects

## Is everything an object?
* A constant?
* An instance of a class?
* A function?
* A module/package?

### Simple types

In [ ]:
isinstance(-1, object)

In [ ]:
(-1).__abs__()

In [ ]:
(-1).__class__

In [ ]:
isinstance(int(-1), object)

### What about functions?

In [ ]:
def test():
    print("test")

isinstance (test, object)

In [ ]:
test.__class__

### Modules?

In [ ]:
import math
isinstance(math, object)

In [ ]:
type(math)

### Other stuff?

In [ ]:
isinstance(..., object)

In [ ]:
type(...)

In [ ]:
isinstance(__debug__, object)

In [ ]:
type(__debug__)

In [ ]:
isinstance(None, object)

In [ ]:
type(None)

### Operators and reserved words?

In [ ]:
# +-*/%//,.[]: 
isinstance(-, object)

In [ ]:
# in not if elif else for while def class...  
isinstance(in, object)

## Looks like almost everything is an object...
### and an instance of a class... 

 so... 

**what about classes?** 

### Sample class

In [ ]:
class Duck:
    pass

In [ ]:
donald = Duck()
isinstance(donald, object)

In [ ]:
isinstance(Duck, object)

In [ ]:
type(Duck)

### `object` and `type`

In [ ]:
type(object)

In [ ]:
type(type)

In [ ]:
isinstance(type, object)

In [ ]:
isinstance(object, type)

In [ ]:
object == type

In [ ]:
issubclass(type, object)

In [ ]:
issubclass(object, type)

### So...in Python, 

* **everything\* is an instance of `object`**, including classes (i.e. `type`) and `object` itself.
* **`object` is an instance of `type`.** 
* **`class` == `type`.**

\* other than operators, punctuation, and reserved words

![Image](mc_escher_drawing_hands_1948.jpg)

## Classes

**But how do classes and instances work?**

### What's a class?

* an object of type `type`
* created with... 
   * `class` keyword
   * class name


In [ ]:
class Duck:
    pass

dir(Duck)

(But most of those are actually in `object`)

In [ ]:
Duck.__dict__

### Instance Methods

* `def` keyword and method name
* parameter reference to instance (self)

In [ ]:
class Duck:
    def hello(self):
        print(f"hello, I'm a {self.__class__.__name__}")

donald = Duck()
donald.hello()

### Instance Data

* data attributes of an instance
* instance referred to as “self”
* “self” is just convention (we could use any legal identifier)

In [ ]:
class Duck:
    def __init__(self, name="a duck", sound='quack'):
        self.name = name
        self.sound = sound
    
    def hello(self):
        print(f"{self.sound}, I'm a {self.__class__.__name__} named {self.name}")

In [ ]:
donald = Duck("Donald")
donald.hello()

In [ ]:
Duck.__dict__

In [ ]:
print(donald)
donald.__dict__

### The other way to make a class

* if we can make an `int` with `int()`, can we make a class (`type`) with `type()`?


In [ ]:
def init_funct(self, name):
    self.name = name

def str_funct(self):
    return f"I am a {self.__class__.__name__} object named {self.name}"

namespace = {'__init__': init_funct, 
             '__str__':str_funct, 
             'x':"value for x"}

namespace

In [ ]:
Scratch = type('Scratch', (), namespace)
print(Scratch)
Scratch.__dict__

In [ ]:
s = Scratch('my_scratch')
print(s)
s.__dict__

### Classes can be created using <br>`type(<classname>, <bases>, <dict>)`

### More on methods

* an instance method is “bound”, i.e. it is called on an instance
* `a_class.method(an_instance, param) == an_instance.method(param)`

In [ ]:
class Duck:
    def __init__(self, name="a duck", sound='quack'):
        self.name = name
        self.sound = sound
    def hello(self):
        return f"I'm a {self.__class__.__name__} named {self.name}"
    def goodbye(self):
        return f"goodbye from the real, true goodbye method for {self.name}"
    
donald = Duck("Donald")
print(donald.hello())
print(donald.goodbye())

In [ ]:
print(Duck.hello(donald))
print(Duck.goodbye(donald))

In [ ]:
# remember the Scratch object 's' from above?
print(Duck.hello(s))
print(Duck.goodbye(s))

### What about changing Duck?



In [ ]:
class Duck:
    def __init__(self, name="a duck", sound='quack'):
        self.name = name
        self.sound = sound
    
    def hello(self):
        return f"{self.sound}, I'm a {self.__class__.__name__} named {self.name}"
    def goodbye(self):
        return f"goodbye from the real, true goodbye method for {self.name}"
    
def goodbye(thing):
    return f"goodbye (function) from {thing.name}"

donald = Duck("Donald")

print(Duck.goodbye(donald))

print(goodbye(donald))

In [ ]:
old_goodbye = Duck.goodbye

Duck.goodbye = goodbye

daisy = Duck("Daisy")

print(daisy.goodbye())

In [ ]:
print(donald.goodbye())


In [ ]:
del Duck.goodbye

daisy.goodbye()

In [ ]:
Duck.goodbye = old_goodbye

print(daisy.goodbye())

### Bound methods are in the class, not the instance

#### Changinnng the class on the fly is "monkeypatching" 

It can make things complicated, if not impossible, debug, so **don't try this at home!**

(or do, I won't stop you, but be warned... here be dragons)

**BUT**

This shows how Python classes work, and that they are not immutable patterns... 

## Metaclasses

* In Python everything is an object, even classes
* Suppose we inherited from `type` and changed a few things?
* We could define classes that behave differently
* e.g. classes that automatically register themselves when instantiated


### Sample metaclass

In [ ]:
class MyMeta(type):
    def __new__(meta, name, bases, dct):
        print('-----------------------------------')
        print(f"Allocating memory for class {name}")
        print(f"{meta=}")
        print(f"{bases=}")
        print(f"{dct=}")
        return super().__new__(meta, name, bases, dct)
    def __init__(cls, name, bases, dct):
        print('-----------------------------------')
        print(f"Initializing class {name}")
        print(f"{cls=}")
        print(f"{bases=}")
        print(f"{dct=}")
        super().__init__(name, bases, dct)

In [ ]:
class MyKlass(metaclass=MyMeta):

    def foo(self, param):
        pass

    barattr = 2

In [ ]:
MyKlass2 = MyMeta('MyKlass2', (),{})
MyKlass2.__dict__

In [ ]:
type(MyMeta)

In [ ]:
type(MyKlass)

In [ ]:
isinstance(MyKlass2, type)

### Another example, a singleton

We could implement a singleton *class*...

But this creates a new *type* of class that **only** creates singleton classes... 

In [ ]:
class ClassSngl(type):
    def __call__(cls, *args, **kwds):
        print(f"Calling {cls.__name__}")
        if not hasattr(cls, "instance"):
            cls.instance = type.__call__(cls, *args, **kwds)

        return cls.instance

In [ ]:
class Sngltn(metaclass=ClassSngl):

    def __init__(self, *args, **kwds):
        print(f"Creating {self.__class__.__name__} instance {self} ")

In [ ]:
single_1 = Sngltn("foo", (), {'a':1, 'b':2})

In [ ]:
single_2 = Sngltn("foo", (), {})

In [ ]:
print(id(single_1))
print(id(single_2))

### Metaclasses can create variations on how classes behave.

Again, I'm **not** recomending you try this at home... other than for experimentation...

The standard wisdom about metaclasses is that if you don't know why you really need them, you probably don't need them. 

## Conclusions

* Everything (almost) is an object
* Classes are objects, too
* Classes can be altered dynamically
* Metaclasses can change how classes behave

### Questions?


## Abstract and outline

"Everything in Python is an object." This is a profound truth about Python, but what does it mean? Is literally EVERYTHING an object? And what is an object anyway? Are objects the same as instances of a class? How do classes and types really work in Python? And what do metaclasses have to do with anything?

In fact, the answers to these questions are probably not what you think they are - Python's approach to objects is different from most other languages in sometimes surprising ways. 

This talk will use simple live coded examples to explore how objects work in Python and clear up several common misconceptions and misunderstandings about how objects and instances, classes and types, and metaclasses all work together. 

Be warned - you are likely to be surprised when you learn the truth about objects in Python.

1. Introduction - 3 min
2. Is everything really  an object? - 5 min
3. class = type - 5 min
    1. What is the type of a class?
    2. Constructing a class using type()
5. What is an instance? - 5 min
     1. How is an instance connected to a class?
     2. Bound methods and monkeypatching
6. Type vs. metaclass - 5 min
     1. Changing how a class is made/behaves
7. Conclusion/Questions - 5 min